In [1]:
import numpy as np
import pandas as pd
import networkx as nx
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import sklearn
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [2]:
edges = pd.read_csv("berlin_digraph_edges.csv")
nodes = pd.read_csv("berlin_digraph_nodes.csv")

#### Rename the source and target columns to 'source' and 'target', respectively

In [3]:
edges = edges.rename(columns = {'Unnamed: 0':'source', 'Unnamed: 1':'target'})

In [4]:
edges.columns

Index(['source', 'target', 'osmid', 'name', 'highway', 'maxspeed', 'oneway',
       'length', 'geometry', 'hour_of_day', 'speed_kph_mean',
       'speed_kph_stddev', 'est_width'],
      dtype='object')

In [5]:
edges.shape

(9012, 13)

In [6]:
edges.head(1)

,source,target,osmid,name,highway,maxspeed,oneway,length,geometry,hour_of_day,speed_kph_mean,speed_kph_stddev,est_width
0,172546,530313,24295532,Emmentaler Straße,secondary,50,False,29.733,"LINESTRING (13.3522545 52.567738, 13.3526642 5...",9,45.276,6.04,NaN


#### Drop 'osmid', 'name', 'geometry' columns

These columns are not predictive of mean speed

In [7]:
edges = edges[['source', 'target', 'highway', 'maxspeed', 'oneway', 'length', 'hour_of_day', 'speed_kph_mean', 'speed_kph_stddev', 'est_width']]
edges.head()

,source,target,highway,maxspeed,oneway,length,hour_of_day,speed_kph_mean,speed_kph_stddev,est_width
0,172546,530313,secondary,50,False,29.733,9,45.276,6.040,NaN
1,172546,28345850,secondary,50,True,64.155,9,14.532,14.245,NaN
2,530313,172546,secondary,50,False,29.733,7,35.007,11.342,NaN
3,28345850,35244866,secondary,50,True,19.106,8,35.756,11.516,NaN
4,28345850,29492967,secondary,50,True,197.683,8,46.770,5.457,NaN


#### All but two values in the 'est_width' column are null, so drop that column too

In [8]:
edges['est_width'].isnull().sum()

9010

In [9]:
edges = edges[['source', 'target', 'highway', 'maxspeed', 'oneway', 'length', 'hour_of_day', 'speed_kph_mean', 'speed_kph_stddev']]
edges.head()

,source,target,highway,maxspeed,oneway,length,hour_of_day,speed_kph_mean,speed_kph_stddev
0,172546,530313,secondary,50,False,29.733,9,45.276,6.040
1,172546,28345850,secondary,50,True,64.155,9,14.532,14.245
2,530313,172546,secondary,50,False,29.733,7,35.007,11.342
3,28345850,35244866,secondary,50,True,19.106,8,35.756,11.516
4,28345850,29492967,secondary,50,True,197.683,8,46.770,5.457


In [10]:
edges.dtypes

source                int64
target                int64
highway              object
maxspeed             object
oneway                 bool
length              float64
hour_of_day           int64
speed_kph_mean      float64
speed_kph_stddev    float64
dtype: object

#### 250 rows have a null 'maxspeed' value

In [11]:
edges['maxspeed'].isnull().sum()

250

In [12]:
edges['maxspeed'].value_counts()

50              6120
30              2444
60               101
10                43
80                32
20                18
['50', '30']       2
40                 1
70                 1
Name: maxspeed, dtype: int64

#### Replace nulls with the mode 'maxspeed' value

In [13]:
edges['maxspeed'].fillna(edges['maxspeed'].mode()[0], inplace=True)

In [14]:
edges['maxspeed'].isnull().sum()

0

In [15]:
edges.dtypes

source                int64
target                int64
highway              object
maxspeed             object
oneway                 bool
length              float64
hour_of_day           int64
speed_kph_mean      float64
speed_kph_stddev    float64
dtype: object

In [16]:
edges['maxspeed'].value_counts()

50              6370
30              2444
60               101
10                43
80                32
20                18
['50', '30']       2
40                 1
70                 1
Name: maxspeed, dtype: int64

#### Get rid of the two annoying '['50', '30']' values and replace them with '50'

In [17]:
edges['maxspeed'] = edges['maxspeed'].replace("['50', '30']", '50')

In [18]:
edges['maxspeed'].value_counts()

50    6372
30    2444
60     101
10      43
80      32
20      18
40       1
70       1
Name: maxspeed, dtype: int64

#### Convert the 'maxspeed' column from categorical to integer

In [19]:
edges['maxspeed'] = edges['maxspeed'].astype(str).astype(int)

In [20]:
edges.dtypes

source                int64
target                int64
highway              object
maxspeed              int32
oneway                 bool
length              float64
hour_of_day           int64
speed_kph_mean      float64
speed_kph_stddev    float64
dtype: object

#### One-hot encode the 'highway' categorical column

In [21]:
modeldf = pd.get_dummies(edges)
modeldf.head()

,source,target,maxspeed,oneway,length,hour_of_day,speed_kph_mean,speed_kph_stddev,"highway_['residential', 'living_street']",highway_living_street,...,highway_motorway_link,highway_primary,highway_primary_link,highway_residential,highway_secondary,highway_secondary_link,highway_tertiary,highway_tertiary_link,highway_trunk_link,highway_unclassified
0,172546,530313,50,False,29.733,9,45.276,6.040,0,0,...,0,0,0,0,1,0,0,0,0,0
1,172546,28345850,50,True,64.155,9,14.532,14.245,0,0,...,0,0,0,0,1,0,0,0,0,0
2,530313,172546,50,False,29.733,7,35.007,11.342,0,0,...,0,0,0,0,1,0,0,0,0,0
3,28345850,35244866,50,True,19.106,8,35.756,11.516,0,0,...,0,0,0,0,1,0,0,0,0,0
4,28345850,29492967,50,True,197.683,8,46.770,5.457,0,0,...,0,0,0,0,1,0,0,0,0,0


In [22]:
modeldf.dtypes

source                                        int64
target                                        int64
maxspeed                                      int32
oneway                                         bool
length                                      float64
hour_of_day                                   int64
speed_kph_mean                              float64
speed_kph_stddev                            float64
highway_['residential', 'living_street']      uint8
highway_living_street                         uint8
highway_motorway                              uint8
highway_motorway_link                         uint8
highway_primary                               uint8
highway_primary_link                          uint8
highway_residential                           uint8
highway_secondary                             uint8
highway_secondary_link                        uint8
highway_tertiary                              uint8
highway_tertiary_link                         uint8
highway_trun

#### Drop the 'speed_kph_stddev' column because it's a function of 'speed_kph_mean'

In [23]:
modeldf = modeldf.drop(['speed_kph_stddev'], axis=1)

In [24]:
modeldf.dtypes

source                                        int64
target                                        int64
maxspeed                                      int32
oneway                                         bool
length                                      float64
hour_of_day                                   int64
speed_kph_mean                              float64
highway_['residential', 'living_street']      uint8
highway_living_street                         uint8
highway_motorway                              uint8
highway_motorway_link                         uint8
highway_primary                               uint8
highway_primary_link                          uint8
highway_residential                           uint8
highway_secondary                             uint8
highway_secondary_link                        uint8
highway_tertiary                              uint8
highway_tertiary_link                         uint8
highway_trunk_link                            uint8
highway_uncl

In [25]:
y = modeldf['speed_kph_mean']

In [26]:
X = modeldf.drop(['speed_kph_mean'], axis=1)

In [27]:
X.head()

,source,target,maxspeed,oneway,length,hour_of_day,"highway_['residential', 'living_street']",highway_living_street,highway_motorway,highway_motorway_link,highway_primary,highway_primary_link,highway_residential,highway_secondary,highway_secondary_link,highway_tertiary,highway_tertiary_link,highway_trunk_link,highway_unclassified
0,172546,530313,50,False,29.733,9,0,0,0,0,0,0,0,1,0,0,0,0,0
1,172546,28345850,50,True,64.155,9,0,0,0,0,0,0,0,1,0,0,0,0,0
2,530313,172546,50,False,29.733,7,0,0,0,0,0,0,0,1,0,0,0,0,0
3,28345850,35244866,50,True,19.106,8,0,0,0,0,0,0,0,1,0,0,0,0,0
4,28345850,29492967,50,True,197.683,8,0,0,0,0,0,0,0,1,0,0,0,0,0


#### Split the dataframe into train, test and split

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=13)

#### Fit a linear regression model

In [29]:
regr = linear_model.LinearRegression().fit(X_train, y_train)
y_pred = regr.predict(X_test)

In [30]:
# The coefficients
# carried over from a homework notebook; not super crucial
print('Coefficients: \n', regr.coef_)

Coefficients: 
 [ 1.41098388e-11  5.87522070e-10  1.87031664e-01  5.45055685e-02
  2.76504646e-02 -5.71075869e-01 -5.82693157e+00 -1.30350125e+01
  2.56613358e+01  4.80637404e+00  5.47530823e+00 -3.51180273e+00
 -4.60293786e+00  3.29265596e+00 -5.32159858e+00  6.47695984e-01
 -6.72361728e+00  0.00000000e+00 -8.61469436e-01]


In [31]:
# Mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(y_test, y_pred))

Mean squared error: 72.89


#### Can't get the plot to work, but not super important

In [32]:
# # Plot outputs
# plt.scatter(X_test, y_test, color='black')
# plt.plot(X_test, y_pred, color='blue', linewidth=3)

# plt.xticks(())
# plt.yticks(())

# plt.show()

In [33]:
print(X_test.shape, y_test.shape, y_pred.shape)

(2253, 19) (2253,) (2253,)


#### Fit a random forest regression model

In [34]:
RFregressor = RandomForestRegressor(n_estimators = 100, random_state = 0).fit(X_train, y_train)
y_pred = RFregressor.predict(X_test)

In [35]:
# Mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(y_test, y_pred))

Mean squared error: 68.52


## Now I'll try dropping the two node columns (source, target) to see if the MSE improves

In [36]:
modeldf = pd.get_dummies(edges)
modeldf.head()
modeldf = modeldf.drop(['speed_kph_stddev', 'source', 'target'], axis=1)
y = modeldf['speed_kph_mean']
X = modeldf.drop(['speed_kph_mean'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=13)

In [37]:
X.head()

,maxspeed,oneway,length,hour_of_day,"highway_['residential', 'living_street']",highway_living_street,highway_motorway,highway_motorway_link,highway_primary,highway_primary_link,highway_residential,highway_secondary,highway_secondary_link,highway_tertiary,highway_tertiary_link,highway_trunk_link,highway_unclassified
0,50,False,29.733,9,0,0,0,0,0,0,0,1,0,0,0,0,0
1,50,True,64.155,9,0,0,0,0,0,0,0,1,0,0,0,0,0
2,50,False,29.733,7,0,0,0,0,0,0,0,1,0,0,0,0,0
3,50,True,19.106,8,0,0,0,0,0,0,0,1,0,0,0,0,0
4,50,True,197.683,8,0,0,0,0,0,0,0,1,0,0,0,0,0


#### Fit a linear regression model

In [38]:
regr = linear_model.LinearRegression().fit(X_train, y_train)
y_pred = regr.predict(X_test)

In [39]:
# Mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(y_test, y_pred))

Mean squared error: 73.37


#### Fit a random forest regression model

In [40]:
RFregressor = RandomForestRegressor(n_estimators = 100, random_state = 0).fit(X_train, y_train)
y_pred = RFregressor.predict(X_test)

In [41]:
# Mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(y_test, y_pred))

Mean squared error: 86.69
